In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from mord import LogisticAT  # Regresión logística ordinal
import seaborn as sns
import matplotlib.pyplot as plt

# Escalamiento de variables
from sklearn.preprocessing import MinMaxScaler

In [34]:
raw_data = pd.read_excel("./DATA DE SATISFACCIÓN HOSPITAL APP_SESIÓN 03.xlsx", 
                         sheet_name = "SAT HOSP_FEATURE")

In [35]:
# raw_data = raw_data[raw_data["hospital"] == 1]
raw_data.drop(columns = ["hospital","caso","rango_edad"], inplace = True)

raw_data.loc[(raw_data["Tipo_hospitalización"] == 3) & (raw_data["sexo"].isna()), "sexo"] = 2
raw_data.loc[(raw_data["Tipo_hospitalización"] == 1) & (raw_data["sexo"].isna()), "sexo"] = 1
raw_data.loc[(raw_data["Tipo_hospitalización"] == 2) & (raw_data["sexo"].isna()), "sexo"] = 1

In [36]:

edad_median = raw_data["edad"].median()

In [37]:
# Imputando la mediana para la fila en la cual no se indicaba el rango de edad
# la mediana asiganda es de la data antes de la imputación de las edades
raw_data.loc[(raw_data["edad"].isna()), "edad"] = edad_median


In [38]:
# Corrigiendo valores atípicos en la variable "sat8"
# La variable "sat8" representa una escala de satisfacción del 1 al 5
sat8_moda = raw_data["sat8"].mode().iloc[0]
raw_data.loc[raw_data["sat8"] > 5, "sat8"] = sat8_moda

In [39]:
# Completando valores nulos
raw_data.loc[ raw_data["sat8"].isna(), "sat8"] = sat8_moda 

In [40]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sexo                  686 non-null    float64
 1   edad                  686 non-null    float64
 2   sat_general           686 non-null    int64  
 3   sat1                  686 non-null    int64  
 4   sat2                  686 non-null    int64  
 5   sat3                  686 non-null    int64  
 6   sat4                  686 non-null    int64  
 7   sat5                  686 non-null    int64  
 8   sat6                  686 non-null    int64  
 9   sat7                  686 non-null    int64  
 10  sat8                  686 non-null    float64
 11  medicion              686 non-null    int64  
 12  N_días_hosp           686 non-null    int64  
 13  Procedencia           686 non-null    int64  
 14  Tipo_hospitalización  686 non-null    int64  
dtypes: float64(3), int64(12

## Escalamiento de variables

In [41]:
# Supongamos que la columna que deseas escalar se llama 'nombre_columna'
scaler_edad = MinMaxScaler()

# Ajustar y transformar los datos
raw_data['edad_escalada'] = scaler_edad.fit_transform(raw_data[['edad']])

In [42]:
# Supongamos que la columna que deseas escalar se llama 'nombre_columna'
scaler_dias_hospital = MinMaxScaler()

# Ajustar y transformar los datos
raw_data['N_dias_hosp_escalada'] = scaler_dias_hospital.fit_transform(raw_data[['N_días_hosp']])


Analisis de satisfaccion

In [43]:
nivel_satisfaccion = raw_data.copy()
# Definimos un diccionario de mapeo
mapeo_satisfaccion = {
    1: 0,
    2: 0,
    3: 0,
    4: 1, # Satisfecho
    5: 1  # Satisfecho
}

nivel_satisfaccion.loc[:,'sat_general_reclasificada'] = nivel_satisfaccion["sat_general"].map(mapeo_satisfaccion)

## Selección de variables

In [44]:
data_selected_variables = nivel_satisfaccion.copy()

In [45]:
data_selected_variables = data_selected_variables.drop(columns = ['sexo', 'medicion', 
                                                                  'Procedencia', 'Tipo_hospitalización',
                                                                  'N_días_hosp','N_dias_hosp_escalada',
                                                                  'edad' # Se elimina la variable edad con tal de solo
                                                                  # considerar la varialbe edad_escalada la cual es más adecuada para
                                                                  # el modelo
                                                                  ])

## Regresión logística ordinal - target: sat_general

In [46]:

# 1. Separar variables predictoras (X) y variable objetivo (y)
X = data_selected_variables.drop(columns=['sat_general', 'sat_general_reclasificada'])
y = data_selected_variables['sat_general_reclasificada']

# 2. Dividir los datos en conjunto de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Entrenar modelo de Regresión Logística Ordinal
model = LogisticAT(alpha=1.0)  # alpha es el parámetro de regularización
model.fit(X_train, y_train)

# 4. Predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# 5. Evaluación del modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nMatriz de Confusión:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.8695652173913043

Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.42      0.55        26
           1       0.88      0.97      0.92       112

    accuracy                           0.87       138
   macro avg       0.83      0.70      0.74       138
weighted avg       0.86      0.87      0.85       138


Matriz de Confusión:
 [[ 11  15]
 [  3 109]]
